In [1]:
from pathlib import Path

from ray.tune.result import TRAINING_ITERATION
from ray.rllib.algorithms.callbacks import DefaultCallbacks
from ray.rllib.core import (
    COMPONENT_LEARNER,
    COMPONENT_LEARNER_GROUP,
    COMPONENT_RL_MODULE,
)
from ray.rllib.core.rl_module.default_model_config import DefaultModelConfig
from ray.rllib.examples.envs.classes.multi_agent import MultiAgentPendulum
from ray.rllib.utils.metrics import (
    ENV_RUNNER_RESULTS,
    EPISODE_RETURN_MEAN,
    NUM_ENV_STEPS_SAMPLED_LIFETIME,
)
from ray.rllib.utils.numpy import convert_to_numpy
from ray.rllib.utils.test_utils import (
    add_rllib_example_script_args,
    check,
    run_rllib_example_script_experiment,
)
from ray.tune.registry import get_trainable_cls, register_env

from pettingzoo.sisl import waterworld_v4

from ray.rllib.core.rl_module.default_model_config import DefaultModelConfig
from ray.rllib.core.rl_module.multi_rl_module import MultiRLModuleSpec
from ray.rllib.core.rl_module.rl_module import RLModuleSpec
from ray.rllib.env.wrappers.pettingzoo_env import PettingZooEnv
from ray.rllib.utils.test_utils import (
    add_rllib_example_script_args,
    run_rllib_example_script_experiment,
)
from ray.tune.registry import get_trainable_cls, register_env

2025-06-24 15:09:21,665	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-06-24 15:09:21,884	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [24]:
import sys
sys.argv = [
    'notebook_script.py',
    '--enable-new-api-stack',
    '--num-agents=2',
    # '--no-tune',
    '--checkpoint-at-end',
    '--stop-reward=200.0',

]
print("参数设置完成")

参数设置完成


In [25]:
parser = add_rllib_example_script_args(
    default_iters=2,
    default_timesteps=10000,
    default_reward=0.0,
)

args = parser.parse_args()

assert args.num_agents > 0, "Must set --num-agents > 0 when running this script!"
assert (
    args.enable_new_api_stack
), "Must set --enable-new-api-stack when running this script!"

print(f"参数解析完成: num_agents={args.num_agents}, algo={args.algo}")


参数解析完成: num_agents=2, algo=PPO


In [26]:
# Cell 4: 注册环境和配置算法
# Here, we use the "Agent Environment Cycle" (AEC) PettingZoo environment type.
# For a "Parallel" environment example, see the rock paper scissors examples
# in this same repository folder.
# Here, we use the "Agent Environment Cycle" (AEC) PettingZoo environment type.
# For a "Parallel" environment example, see the rock paper scissors examples
# in this same repository folder.
register_env("env", lambda _: PettingZooEnv(waterworld_v4.env()))

# Policies are called just like the agents (exact 1:1 mapping).
policies = {f"pursuer_{i}" for i in range(args.num_agents)}

base_config = (
    get_trainable_cls(args.algo)
    .get_default_config()
    .environment("env")
    .multi_agent(
        policies=policies,
        # Exact 1:1 mapping from AgentID to ModuleID.
        policy_mapping_fn=(lambda aid, *args, **kwargs: aid),
    )
    .training(
        vf_loss_coeff=0.005,
    )
    .rl_module(
        rl_module_spec=MultiRLModuleSpec(
            rl_module_specs={p: RLModuleSpec() for p in policies},
        ),
        model_config=DefaultModelConfig(vf_share_layers=True),
    )
)

# run_rllib_example_script_experiment(base_config, args)
results = run_rllib_example_script_experiment(base_config, args, keep_ray_up=True)
print("环境注册和配置完成")

2025-06-24 15:17:42,603	INFO worker.py:1917 -- Started a local Ray instance.


== Status ==
Current time: 2025-06-24 15:17:43 (running for 00:00:00.11)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-06-24_15-17-42_041583_547330/artifacts/2025-06-24_15-17-43/PPO_2025-06-24_15-17-43/driver_artifacts
Number of trials: 1/1 (1 PENDING)




(PPO pid=555405) 2025-06-24 15:17:44,553	WARNING algorithm_config.py:5014 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
(MultiAgentEnvRunner pid=555491) 2025-06-24 15:17:46,291	WARNING deprecation.py:50 -- DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=..)` instead. This will raise an error in the future!
(PPO pid=555405) Install gputil for GPU system monitoring.


== Status ==
Current time: 2025-06-24 15:17:48 (running for 00:00:05.21)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-06-24_15-17-42_041583_547330/artifacts/2025-06-24_15-17-43/PPO_2025-06-24_15-17-43/driver_artifacts
Number of trials: 1/1 (1 RUNNING)




Trial name,env_runner_group,env_runners,fault_tolerance,learners,num_env_steps_sampled_lifetime,num_training_step_calls_per_iteration,perf,timers
PPO_env_f0114_00000,{'actor_manager_num_outstanding_async_reqs': 0},"{'connector_pipeline_timer': np.float64(0.0003068314981646836), 'episode_return_min': -313.726871349234, 'env_reset_timer': np.float64(0.0016084969975054264), 'rlmodule_inference_timer': np.float64(9.399915656965589e-05), 'env_step_timer': np.float64(0.0003813357210763395), 'episode_return_mean': -261.8934658410173, 'env_to_module_connector': {'timers': {'connectors': {'add_states_from_episodes_to_batch': np.float64(3.952719020689228e-06), 'batch_individual_items': np.float64(1.2691392393109051e-05), 'add_observations_from_episodes_to_batch': np.float64(1.6158807069575928e-05), 'numpy_to_tensor': np.float64(2.2054553432598536e-05), 'add_time_dim_to_batch_and_zero_pad': np.float64(6.8665170436234445e-06), 'agent_to_module_mapping': np.float64(3.373805059319909e-06)}}, 'connector_pipeline_timer': np.float64(0.00010726873230809766)}, 'module_to_env_connector': {'timers': {'connectors': {'get_actions': np.float64(8.311198329903125e-05), 'normalize_and_clip_actions': np.float64(3.5616559864388356e-05), 'tensor_to_numpy': np.float64(3.237276597851163e-05), 'remove_single_ts_time_rank_from_batch': np.float64(1.09656278159291e-06), 'listify_data_for_vector_env': np.float64(6.0032374115883e-06), 'un_batch_to_individual_items': np.float64(1.3689150840869613e-05), 'module_to_agent_unmapping': np.float64(2.748299035598287e-06)}}, 'connector_pipeline_timer': np.float64(0.00022956203345865544)}, 'timers': {'connectors': {'add_time_dim_to_batch_and_zero_pad': np.float64(1.4111501513980329e-05), 'agent_to_module_mapping': np.float64(5.716006853617728e-06), 'add_states_from_episodes_to_batch': np.float64(5.333502485882491e-06), 'numpy_to_tensor': np.float64(4.365100176073611e-05), 'batch_individual_items': np.float64(2.2173000616021454e-05), 'add_observations_from_episodes_to_batch': np.float64(2.950499765574932e-05)}}, 'episode_len_mean': 1000.0, 'sample': np.float64(1.974725075719689), 'num_agent_steps_sampled': {'pursuer_0': 2000.0, 'pursuer_1': 2004.0}, 'num_episodes': 4.0, 'episode_return_max': -173.29817059173007, 'num_episodes_lifetime': 8.0, 'env_to_module_sum_episodes_length_in': np.float64(891.0423566894266), 'num_module_steps_sampled_lifetime': {'pursuer_0': 4000.0, 'pursuer_1': 4008.0}, 'num_env_steps_sampled_lifetime': 8000.0, 'module_episode_returns_mean': {'pursuer_0': -85.1942479220129, 'pursuer_1': -176.6992179190044}, 'env_to_module_sum_episodes_length_out': np.float64(891.0423566894266), 'num_agent_steps_sampled_lifetime': {'pursuer_0': 4000.0, 'pursuer_1': 4008.0}, 'episode_len_max': 1000, 'weights_seq_no': 1.0, 'episode_len_min': 1000, 'num_module_steps_sampled': {'pursuer_0': 2000.0, 'pursuer_1': 2004.0}, 'episode_duration_sec_mean': 0.975714623253225, 'agent_episode_returns_mean': {'pursuer_0': -85.1942479220129, 'pursuer_1': -176.6992179190044}, 'agent_steps': {'pursuer_0': 500.0, 'pursuer_1': 500.0}, 'num_env_steps_sampled': 4000.0, 'time_between_sampling': np.float64(2.769492552499287), 'num_env_steps_sampled_lifetime_throughput': np.float64(1117.7133516949234)}","{'num_healthy_workers': 2, 'num_remote_worker_restarts': 0}","{'pursuer_1': {'num_trainable_parameters': 129285, 'curr_kl_coeff': 0.45000001788139343, 'vf_explained_var': np.float32(0.004059255), 'mean_kl_loss': np.float32(0.026672682), 'vf_loss': np.float32(9.922472), 'policy_loss': np.float32(-0.0292667), 'weights_seq_no': 2.0, 'default_optimizer_learning_rate': 5e-05, 'curr_entropy_coeff': 0.0, 'entropy': np.float32(2.9062617), 'vf_loss_unclipped': np.float32(917.9136), 'num_module_steps_trained': 60160, 'total_loss': np.float32(0.02834747), 'gradients_default_optimizer_global_norm': np.float32(1.0176111), 'diff_num_grad_updates_vs_sampler_policy': np.float32(1.0), 'module_train_batch_size_mean': 128.0, 'num_module_steps_trained_lifetime': 1203

...

(PPO(env=env; env-runners=2; learners=0; multi-agent=True) pid=555405) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/qrbao/ray_results/PPO_2025-06-24_15-17-43/PPO_env_f0114_00000_0_2025-06-24_15-17-43/checkpoint_000000)
2025-06-24 15:17:56,182	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/qrbao/ray_results/PPO_2025-06-24_15-17-43' in 0.0131s.


== Status ==
Current time: 2025-06-24 15:17:56 (running for 00:00:13.18)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-06-24_15-17-42_041583_547330/artifacts/2025-06-24_15-17-43/PPO_2025-06-24_15-17-43/driver_artifacts
Number of trials: 1/1 (1 TERMINATED)
+---------------------+------------+---------------------+--------+------------------+------+-------------------+--------------------+--------------------+
| Trial name          | status     | loc                 |   iter |   total time (s) |   ts |   combined return |   return pursuer_0 |   return pursuer_1 |
|---------------------+------------+---------------------+--------+------------------+------+-------------------+--------------------+--------------------|
| PPO_env_f0114_00000 | TERMINATED | 192.168.0.25:555405 |      2 |          9.11264 | 8000 |          -261.893 |           -85.1942 |           -176.699 |
+------------------

2025-06-24 15:17:56,862	INFO tune.py:1041 -- Total run time: 13.86 seconds (13.16 seconds for the tuning loop).


环境注册和配置完成


(raylet) [2025-06-24 15:18:02,511 E 553928 553958] (raylet) file_system_monitor.cc:116: /tmp/ray/session_2025-06-24_15-17-42_041583_547330 is over 95% full, available space: 5.48897 GB; capacity: 456.175 GB. Object creation will fail if spilling is required.
(raylet) [2025-06-24 15:18:12,521 E 553928 553958] (raylet) file_system_monitor.cc:116: /tmp/ray/session_2025-06-24_15-17-42_041583_547330 is over 95% full, available space: 5.48897 GB; capacity: 456.175 GB. Object creation will fail if spilling is required.
(PPO pid=555728) 2025-06-24 15:18:20,652	WARNING algorithm_config.py:5014 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
(MultiAgentEnvRunner pid=555800) 2025-06-24 15:18:22

In [15]:

class LoadP0OnAlgoInitCallback(DefaultCallbacks):
    def on_algorithm_init(self, *, algorithm, **kwargs):
        module_p0 = algorithm.get_module("pursuer_0")
        weight_before = convert_to_numpy(next(iter(module_p0.parameters())))
        algorithm.restore_from_path(
            p_0_module_state_path,
            component=(
                COMPONENT_LEARNER_GROUP
                + "/"
                + COMPONENT_LEARNER
                + "/"
                + COMPONENT_RL_MODULE
                + "/pursuer_0"
            ),
        )
        # Make sure weights were updated.
        weight_after = convert_to_numpy(next(iter(module_p0.parameters())))
        check(weight_before, weight_after, false=True)

base_config.callbacks(LoadP0OnAlgoInitCallback)

# Define stopping criteria.
stop = {
    f"{ENV_RUNNER_RESULTS}/{EPISODE_RETURN_MEAN}": -800.0,
    f"{ENV_RUNNER_RESULTS}/{NUM_ENV_STEPS_SAMPLED_LIFETIME}": 100000,
    TRAINING_ITERATION: 100,
}


In [16]:

print("开始第二次训练（使用预训练权重）...")
# Run the experiment again with the restored MultiRLModule.
final_results = run_rllib_example_script_experiment(base_config, args, stop=stop)
print("训练完成！")


开始第二次训练（使用预训练权重）...


2025-06-24 15:10:13,376	INFO worker.py:1917 -- Started a local Ray instance.


== Status ==
Current time: 2025-06-24 15:10:13 (running for 00:00:00.11)
Using FIFO scheduling algorithm.
Logical resource usage: 1.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-06-24_15-10-12_812887_547330/artifacts/2025-06-24_15-10-13/PPO_2025-06-24_15-10-13/driver_artifacts
Number of trials: 1/1 (1 PENDING)




(PPO pid=550911) 2025-06-24 15:10:15,320	WARNING algorithm_config.py:5014 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
2025-06-24 15:10:15,418	ERROR tune_controller.py:1331 -- Trial task failed for trial PPO_env_e44ce_00000
Traceback (most recent call last):
  File "/home/qrbao/anaconda3/envs/rllib/lib/python3.10/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/home/qrbao/anaconda3/envs/rllib/lib/python3.10/site-packages/ray/_private/auto_init_hook.py", line 22, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/home/qrbao/anaconda3/envs/rllib/lib/python3.10/site-packages/ray/_private/client_mo

Trial name
PPO_env_e44ce_00000


2025-06-24 15:10:15,422	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/qrbao/ray_results/PPO_2025-06-24_15-10-13' in 0.0020s.
2025-06-24 15:10:15,423	ERROR tune.py:1037 -- Trials did not complete: [PPO_env_e44ce_00000]
2025-06-24 15:10:15,423	INFO tune.py:1041 -- Total run time: 1.66 seconds (1.65 seconds for the tuning loop).
2025-06-24 15:10:15,424	WARNING experiment_analysis.py:180 -- Failed to fetch metrics for 1 trial(s):
- PPO_env_e44ce_00000: FileNotFoundError('Could not fetch metrics for PPO_env_e44ce_00000: both result.json and progress.csv were not found at /home/qrbao/ray_results/PPO_2025-06-24_15-10-13/PPO_env_e44ce_00000_0_2025-06-24_15-10-13')


== Status ==
Current time: 2025-06-24 15:10:15 (running for 00:00:01.66)
Using FIFO scheduling algorithm.
Logical resource usage: 0/24 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-06-24_15-10-12_812887_547330/artifacts/2025-06-24_15-10-13/PPO_2025-06-24_15-10-13/driver_artifacts
Number of trials: 1/1 (1 ERROR)
+---------------------+----------+-------+
| Trial name          | status   | loc   |
|---------------------+----------+-------|
| PPO_env_e44ce_00000 | ERROR    |       |
+---------------------+----------+-------+
Number of errored trials: 1
+---------------------+--------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name          |   # failures | error file                                                                                                                                                 

RuntimeError: Running the example script resulted in one or more errors! [EnvError("The env string you provided ('env') is:\na) Not a supported or -installed environment.\nb) Not a tune-registered environment creator.\nc) Not a valid env class string.\n\nTry one of the following:\na) For Atari support: `pip install gym[atari] autorom[accept-rom-license]`.\n   For PyBullet support: `pip install pybullet`.\nb) To register your custom env, do `from ray import tune;\n   tune.register_env('[name]', lambda cfg: [return env obj from here using cfg])`.\n   Then in your config, do `config.environment(env='[name]').\nc) Make sure you provide a fully qualified classpath, e.g.:\n   `ray.rllib.examples.envs.classes.repeat_after_me_env.RepeatAfterMeEnv`\n")]

In [ ]:

# Cell 7: 显示结果
print("=" * 50)
print("训练结果:")
print(f"最佳checkpoint路径: {final_results.get_best_result().checkpoint.path}")
print("=" * 50)

In [ ]:


    ort_session = None
    print(" ok")
